<a href="https://colab.research.google.com/github/Anil-matcha/LlamaIndex-tutorials/blob/main/LlamaIndex_QA_%2B_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from llama_index import (
    VectorStoreIndex,
    ListIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext)

In [ ]:
!wget https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt
!mkdir data
!mv state_of_the_union.txt data/

In [12]:
documents = SimpleDirectoryReader('./data').load_data()

In [13]:
import os
os.environ["OPENAI_API_KEY"] = "openai-key"
service_context = ServiceContext.from_defaults(chunk_size=1024)
nodes = service_context.node_parser.get_nodes_from_documents(documents)

In [14]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [15]:
list_index = ListIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [16]:
list_query_engine = list_index.as_query_engine(
    response_mode='tree_summarize',
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [17]:
from llama_index.tools.query_engine import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description='Useful for summarization questions related to Paul Graham eassy on What I Worked On.',
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description='Useful for retrieving specific context from Paul Graham essay on What I Worked On.',

)

In [18]:
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ]
)

In [23]:
print(query_engine.query('What is the summary of the document?').response)


In this speech, President Biden calls on Congress to pass legislation to fight inflation, lower the cost of prescription drugs, cut energy costs, end the shutdown of schools and businesses, continue vaccinating the world, and protect communities by investing in crime prevention and community police officers. He also calls for universal background checks and the repeal of the liability shield for gun manufacturers. Finally, he calls for the passage of the Freedom to Vote Act, the John Lewis Voting Rights Act, and the Disclose Act to protect the right to vote.


In [22]:
print(query_engine.query('What is NaTO?').response)


NATO is the North Atlantic Treaty Organization, an intergovernmental military alliance between 30 North American and European countries. It was created after World War II to secure peace and stability in Europe.
